In [1]:
%cd ..

/home/user/tmp/face_toolbox_keras


In [2]:
import json
from os.path import join
from pprint import pprint


# lyrics_file = join('./', 'assets', 'lyrics.json')
from face_toolbox.audio import lyrics

In [16]:
lrc = lyrics.Lyrics()
info = lambda i: print(f'word @ {i}={lrc.words[i]}\t sec={lrc.beat_times[i]}, frame #={lrc.beat_frames[i]}, duration_sec={lrc.beat_delta_sec(i)}, duration_frame_#={lrc.beat_delta_frames(i)}')

In [17]:
for i in range(0, 20, 3):
    info(i)

word @ 0=Buy	 sec=0.7421102040816329, frame #=44, duration_sec=0.7421102040816329, duration_frame_#=12
word @ 3=it,	 sec=1.595456437389767, frame #=95, duration_sec=0.2451898211136294, duration_frame_#=4
word @ 6=fix	 sec=2.307470546737207, frame #=138, duration_sec=0.2558968002015609, duration_frame_#=4
word @ 9=it,	 sec=3.0362789773242564, frame #=182, duration_sec=0.18291509977324294, duration_frame_#=3
word @ 12=mail	 sec=3.697587414965982, frame #=221, duration_sec=0.25651407482993305, duration_frame_#=4
word @ 15=it	 sec=4.445482882086166, frame #=266, duration_sec=0.17209172108843696, duration_frame_#=2
word @ 18=point	 sec=5.120747684324897, frame #=307, duration_sec=0.25562809142775844, duration_frame_#=4


In [ ]:
with open(lyrics_file) as f:
  lyrics_dict = json.load(f)

print(lyrics_dict.keys())

In [ ]:
for idx, line in enumerate(lyrics_dict['lines']):
    print(f"{line}, {lyrics_dict['wordTimings'][idx]}")

In [ ]:
### What we want to do is get a list of [idx, word, time] entries so we know the exact beats

# First, we need a list of all the words. 
# simplest case, split on spaces
all_words = [w for l in lyrics_dict['lines'] for w in l.split()]

# all timings
all_times = [t for l in lyrics_dict['wordTimings'] for t in l]

# indices
all_indices = [i for i in range(0, len(all_times))]

all_beats = list(zip(all_indices, all_words, all_times))


In [ ]:
all_it_beats = [i for i in all_beats if (i[1].lower().startswith('it'))]
print(len(all_it_beats))
pprint(all_it_beats)

In [ ]:
class LyricsIterator:
    def __init__(self, data):
        self._idx = 0
        self._data = data
    
    def __next__(self):
        if (self._idx < len(self._data.emph)):
            res = self._data.emph[self._idx]
            self._idx += 1
            return res
        raise StopIteration

class Lyrics:
    def __init__(self, path: str):
        with open(path) as f:
            data = json.load(f)
        self.lines       = data['lines']
        self.wordTimings = data['wordTimings']
        self.markers     = data['markers']

        all_words = [w for l in self.lines for w in l.split()]
        all_times = [t for l in self.wordTimings for t in l]
        all_beats = list(zip(all_words, all_times))
        self.emph = [i for i in all_beats if (i[0].lower().startswith('it'))]


    def __iter__(self):
        return LyricsIterator(self)

    @property
    def words(self):
        return [w for l in self.lines for w in l.split()]

    @property
    def beats(self):
        return [t for l in self.wordTimings for t in l]

    def beat_delta_ms(self, idx: int) -> float:
        if (idx < 0 | idx > (len(self.emph) - 1)):
            raise ValueError(f"index {idx} out of bounds")
        elif (idx == 0):
            return self.emph[idx][1]
        elif (idx+1 > len(self.emph)-1):
            return 4.0
        else:
            return self.emph[idx+1][1] - self.emph[idx][1]

    def beat_delta_frames(self, idx: int):
        beatΔ = self.beat_delta_ms(idx)
        frames = int(round(beatΔ * 1000) / 16.6666667)
        return frames


lrc = Lyrics(lyrics_file)

In [ ]:
for idx, (w, t) in enumerate(lrc):
    print(w, t, lrc.beat_delta_ms(idx), lrc.beat_delta_frames(idx))

In [ ]:
t = [1]

In [ ]:
t[2]

In [ ]:
import lyricsgenius
genius = lyricsgenius.Genius('...')
song_id     = search[0]['result']['id']
song        = genius.search_song(song_id)
annotations = genius.get_song_annotations(song_id)
print(annotations[0][1])
for idx, anot in enumerate(annotations):
    print(f'{idx}\n{anot[0]}\n')